In [1]:
import os
import random
from collections import defaultdict
os.chdir("/home/ys/ExplainablePPI")
from experiments.utils import organisms

random.seed(100)

/home/ys/anaconda3/envs/workspace/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_pairs(pair_dir, k):
    pairs = defaultdict(list)
    for orga in organisms:
        #读取pair
        orga_pair_fp = os.path.join(pair_dir, orga+"_test.tsv")
        lines = open(orga_pair_fp, "r").readlines()
        pos_lines = [l for l in lines if l.strip()[-1] == '1']
        pairs[orga] = random.sample(pos_lines, k=k)
    return pairs

def get_seqs(seq_dir, pairs):
    # 读取 seq
    seqs = defaultdict(list)
    for orga in organisms:
        orga_fasta_fp = os.path.join(seq_dir, orga+"_test.fasta")
        lines = open(orga_fasta_fp, "r").readlines()
        orga_seq_dict = dict([l.strip().split() for l in lines])
        pros = set([])
        for p in pairs[orga]:
            fpro, spro, _ = p.strip().split()
            pros.add(fpro)
            pros.add(spro)
        seqs[orga] = [f"{pro}\t{orga_seq_dict[pro]}\n" for pro in pros]
    return seqs

In [3]:
seq_dir = "./data/dscript/processed/seqs"
pair_dir = "./data/dscript/processed/pairs"

#! todo 会选出重复的样本，需要纠正
pairs = get_pairs(pair_dir=pair_dir, k=2000)
train_pairs = {}
test_pairs = {}
for orga in pairs:
    random.shuffle(pairs[orga])
    train_pairs[orga] = pairs[orga][:1600]
    test_pairs[orga] = pairs[orga][1600:]

train_seqs = get_seqs(seq_dir=seq_dir, pairs=train_pairs)
test_seqs = get_seqs(seq_dir=seq_dir, pairs=test_pairs)

os.makedirs("./experiments/10.ppi_aa_freq_prediction/data/train", exist_ok=True)
os.makedirs("./experiments/10.ppi_aa_freq_prediction/data/test", exist_ok=True)

for orga in organisms:
    with open(f"./experiments/10.ppi_aa_freq_prediction/data/test/{orga}.seq", "w") as f:
        f.writelines(test_seqs[orga])
    with open(f"./experiments/10.ppi_aa_freq_prediction/data/test/{orga}.tsv", "w") as f:
        f.writelines(test_pairs[orga])

    with open(f"./experiments/10.ppi_aa_freq_prediction/data/train/{orga}.seq", "w") as f:
        f.writelines(train_seqs[orga])
    with open(f"./experiments/10.ppi_aa_freq_prediction/data/train/{orga}.tsv", "w") as f:
        f.writelines(train_pairs[orga])
